In [1]:
import pandas as pd
import openml

dataset = openml.datasets.get_dataset(44061)
X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)

X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
columns_to_encode = ['X3', 'X4', 'X6']
X = pd.get_dummies(X, columns=columns_to_encode)

C:\Users\elsag\OneDrive\Desktop\Uni\Year 4\Academic\Dissertation\Final Report\Code\scikit-learn-1.2.1\scikit-learn-1.2.1\venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


In [3]:
X = X.astype(float)
y = y.astype(float)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [5]:
from tensorflow import keras

def fit_bnn_model(model, loss, num_epochs, learning_rate, X_train, y_train):
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-8,
                                        clipnorm=1),
        loss=loss,
        metrics=[keras.metrics.MeanSquaredError()],
    )

    model.fit(x=X_train, y=y_train, epochs=num_epochs)

In [6]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_probability as tfp

In [12]:
def bnn_model(hidden_units, input_shape, train_size, activation):
    # Create the input layer
    input_layer = layers.Input(shape=input_shape)
    features = input_layer

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for i, units in enumerate(hidden_units):
        if i!= len(hidden_units)-1:
            features = layers.Dense(units, activation=activation)(features)
        else:
            features = tfp.layers.DenseVariational(
                units=units,
                make_prior_fn=bnn.prior,
                make_posterior_fn=bnn.posterior,
                kl_weight=1 / train_size,
                activation=activation,
            )(features)
            features = layers.Dropout(0, 2)(features)

    # create output - mean and std
    distribution_params = layers.Dense(units=tfp.layers.IndependentNormal.params_size(2))(features)
    outputs = tfp.layers.IndependentNormal(2)(distribution_params)

    model = keras.Model(inputs=input_layer, outputs=outputs)

    return model

In [19]:
from uncertainty_analysis import bnn_helper_functions as bnn

hidden_units = [100, 10]
learning_rate = 0.0005
num_epochs = 500
input_shape = X_train.shape[1]
train_size = len(X_train)

predictions = []
for i in range(0, 1):
    model = bnn_model(hidden_units, input_shape, train_size, activation='sigmoid')
    fit_bnn_model(model, bnn.negative_loglikelihood, num_epochs, learning_rate, X_train, y_train)
    predictions.append(model.predict(X_test))

Epoch 1/500
93/93 [==============================] - 47s 25ms/step - loss: 12076.0527 - mean_squared_error: 10254.7451
Epoch 2/500
93/93 [==============================] - 2s 24ms/step - loss: 4024.3499 - mean_squared_error: 10156.4590
Epoch 3/500
93/93 [==============================] - 2s 24ms/step - loss: 2942.1880 - mean_squared_error: 10013.8828
Epoch 4/500
93/93 [==============================] - 2s 24ms/step - loss: 1749.1951 - mean_squared_error: 9903.7832
Epoch 5/500
93/93 [==============================] - 2s 24ms/step - loss: 786.9741 - mean_squared_error: 9686.3867
Epoch 6/500
93/93 [==============================] - 2s 25ms/step - loss: 445.7102 - mean_squared_error: 9464.7549
Epoch 7/500
93/93 [==============================] - 2s 25ms/step - loss: 305.7695 - mean_squared_error: 9248.7246
Epoch 8/500
93/93 [==============================] - 2s 25ms/step - loss: 189.9445 - mean_squared_error: 8967.0518
Epoch 9/500
93/93 [==============================] - 2s 24ms/step - los

In [24]:
 from uncertainty_analysis import project_helper_functions as phf

best_model_results = predictions[phf.bf_get_best_model_idx(predictions, y_test)]
phf.write_bnn_model_results(best_model_results, "merc_bnn_results.txt")